In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
from zipfile import ZipFile
import csv
import numpy as np
from skimage.measure import find_contours, label
from jpype import JArray, JInt

In [2]:
scyjava.config.add_option('-Xmx30g') # this is to assign RAM to the java session
ij = imagej.init('/home/saka/sw/local/fiji2/2023', mode='interactive')
ij.ui().showUI()
ij.getVersion()

'2.14.0/1.54f'

In [3]:
# Dialog window to get the path of the working directory
root = tk.Tk()
root.withdraw()
# Set the path to the directory containing the images
directory_path = filedialog.askdirectory()

In [8]:
w = scyjava.jimport('ij.WindowManager')

In [ ]:
folder_list = os.listdir(directory_path)
for folder in folder_list:
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)
    for image in file_list:
        basename = os.path.basename(image)
        corename = os.path.splitext(basename)[0]
        imp = ij.IJ.openImage(image)
        imp.show()
        composite = f"{corename}_composite.tif"
        ij.IJ.run("Select None")
        ij.IJ.run("Duplicate...", f"title={str(composite)} duplicate channels=1-2")
        ij.IJ.selectWindow(composite)
        output = f"{folder_path}/{composite}"
        image_composite = w.getImage(str(composite))
        ij.IJ.save(image_composite, output)
        ij.py.run_macro('close("*");')

In [12]:
IC = scyjava.jimport('ij.plugin.ImageCalculator')

In [ ]:
for folder in folder_list:
    folder_path = os.path.join(directory_path, folder)
    # Find all the czi files in the folder
    file_pattern = os.path.join(folder_path, "*composite.tif")
    file_list = glob.glob(file_pattern)
    for image in file_list:
        imp = ij.IJ.openImage(image)
        imp.show()
        blue = f"Blue"
        green = f"Green"
        ij.IJ.run("Duplicate...", f"title={str(blue)} duplicate channels=1")
        ij.IJ.run("Duplicate...", f"title={str(green)} duplicate channels=2")
        imp1 = w.getImage(str(blue))
        imp2 = w.getImage(str(green))
        imp3 = IC.run("Subtract create 32-bit stack", "Blue","Green")
        imp4 = IC.run("Add create 32-bit stack", "Blue","Green")
        imp5 = IC.run("Divide create 32-bit stack", imp3, imp4)
        imp5.show()
    

    
nom=getTitle;
	w=getWidth(); 
	selectWindow(nom);
		run("Make Substack...", "channels=1 stack");
		rename("Blue");

	selectWindow(nom);
	run("Make Substack...", "channels=2 stack");
	rename("Green");

	imageCalculator("Subtract create 32-bit stack", "Blue","Green");
	rename("Num");
	imageCalculator("Add create 32-bit stack", "Blue","Green");
	rename("Den");
	imageCalculator("Divide create 32-bit stack", "Num","Den");
	rename(nom + " - GP");
	run("LUTspectrum modif");

In [22]:
image = f"{directory_path}/h37 etoh/Experiment-272_composite.tif"
imp = ij.IJ.openImage(image)
imp.show()
blue = f"Blue"
green = f"Green"
ij.IJ.run("Duplicate...", f"title={str(blue)} duplicate channels=1")
ij.IJ.run("Duplicate...", f"title={str(green)} duplicate channels=2")
imp1 = w.getImage(str(blue))
imp2 = w.getImage(str(green))
imp3 = IC.run(imp1, imp2, "Subtract create 32-bit stack")
imp3.show()
imp4 = IC.run(imp1, imp2, "Add create 32-bit stack")
imp4.show()
imp5 = IC.run(imp3, imp4, "Divide create 32-bit stack")
imp5.show()

In [21]:
ij.py.run_macro('close("*");')

<java object 'org.scijava.script.ScriptModule'>